# RedGrease Demo at RedisConf 2021

Quick demonstration of how to create and run Redis Gears functions, using RedGrease.

## [Demos](#Demos):
1. [The Basics](#1.-The-Basics)
2. [Simple Analytics Query](#2.-Simple-Analytics-Query)
3. [Transaction Stream Processing](#3.-Transaction-Stream-Processing)
4. [Custom Command](#4.-Custom-Command)


# Preparations
Before running the demos, make sure that the prerequisites are met and that the preparation steps have successfully been executed. 
Some preparation steps, particularly the downloads, may take quite some time. 

## 1. Prerequisites
- Python3.7
- Pip
- Docker
- Jupyter

Run the cell below tho validate your prerequisites.

In [1]:
# Run cell to test your environment requirements

import sys
import re
pyver = !{sys.executable} --version  # type: ignore
pipver = !{sys.executable} -m pip --version  # type: ignore
dockver = !docker --version  # type: ignore

if not re.match("Python 3.7", pyver[0]):
    raise SystemExit(f"This demo only supports Python 3.7. You are running {pyver[0]}.")

if not re.match(".*\(python 3.7\)", pipver[0]):
    raise SystemExit("Please install Pip for yout Python 3.7 environment.")

if not re.match("Docker version", dockver[0]):
    raise SystemExit("Please install Docker")

print("Requirements all look good!")

Requirements all look good!


## 2. Python Requirements

Install the Python packages required for the demo:

- `redgrease[client]` - The RedGrease client library for Redis Gears. This is what is being demonstrated.

- `ipywidgets` - Jupyter notebook exetension, for displaying widgets, e.g. buttons, in this notebook.
- `requests` - For downloading content.

Run the cell below to install the requirements.

In [2]:
%%capture reqs_install_output
!{sys.executable} -m pip install redgrease[client] ipywidgets requests
!jupyter nbextension enable --py widgetsnbextension

## 3. Download Datasets
Some of the demos requiere a portion of the [COCO Dataset](https://cocodataset.org) to be uploaded into the Redis Gears Cluster.
The COCO Dataset (Common Objects in Context) is a fairly large set of (~247,000) images and corresponding annotations of what tey are depicting.

### Example:
<img src="coco_example.jpg" > [COCO Example](coco_example.jpg)

```
a man riding a snowboard down a ski slope.
a snowboarder sailing down a snowy hillside on a mountain.
a man is snowboarding past blue markers on a mountain.
a man on a snowboard in the snow.
a man snow boarding in the snow on a slope. 
```


For the demo we will only pre-download the annotations (json), not the images (jpeg), but it is still between 250 - 500 MB of data, depending on which portions you choose.

There are two annotation packages to choose from. 
- **COCO Train/Cal 2014** - Annotations for 124,000 images (241 MB)
- **COCO Train/Val 2017** - Annotations for 123,000 images (241 MB)

Either or both may be used. 
Run the cell below and select using the buttons which dataset(s) to download.

In [3]:
# This code is just for preparation of the demo.
# It is NOT part of the demo itself
#
# Download COCO Annotations 
# Run the cell, then:
# - Validate or modify the Download directory
# - Click the button, or buttons for the annotations to download

import ipywidgets as widgets
import os
import requests

coco_annotations_url = "http://images.cocodataset.org/annotations"
annotations_file_pattern = "annotations_trainval{}.zip"

layout = widgets.Layout(width="30%")
output = widgets.Output()

def get_download_path():
    download_dir = "."
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            download_dir = winreg.QueryValueEx(key, downloads_guid)[0]
    else:
        download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

    return os.path.join(download_dir, "COCO")

download_location = widgets.Text(
    value=get_download_path(),
    placeholder="Download directory",
    description="Directory to download annotations to.",
    layout=layout,
)
display(download_location)

def dl_state(button, downloading=None):
    year = button.value
    annotations_file_name = annotations_file_pattern.format(year)
    destination = os.path.join(download_location.value, annotations_file_name)
    is_downloaded = os.path.isfile(destination)
    button.disabled = is_downloaded or downloading is not None
    if downloading:
        button.description=f"Downloading COCO {year} annotations (241 MB): {downloading}%. Please wait!"
    elif is_downloaded:
        button.description=f"Congrats! COCO {year} annotataions is downloaded!"
    else:
        button.description=f"Download COCO {year} annotations (241 MB)"
    return is_downloaded, annotations_file_name, destination


def download_button_pressed(btn):
    downloaded, file_name, destination = dl_state(btn)
    if downloaded:
        return
    if not os.path.isdir(download_location.value):
        os.mkdir(download_location.value)
    try:
        response = requests.get(
            f"{coco_annotations_url}/{file_name}",
            stream=True
        )
        total_length = response.headers.get('content-length')
        with open(destination, "wb") as f:
            if total_length is None: # no content length header
                dl_state(btn, "???")
                f.write(response.content)
                return
            total_length = int(total_length)
            dl = 0
            for data in response.iter_content(chunk_size=4096):
                dl += len(data)
                f.write(data)
                dl_state(btn, int(100*(dl/total_length)))

    except Exception:
        try:
            os.remove(destination)
        except Exception:
            pass
    finally:
        dl_state(btn)

for year in ["2014", "2017"]:
    download_button = widgets.Button(
        tooltip='Start download of selected datasets into the selected download directory.',
        layout=layout
    )
    download_button.value = year
    dl_state(download_button)
    download_button.on_click(download_button_pressed)
    display(download_button)

display(output)

Text(value='/home/anders/Downloads/COCO', description='Directory to download annotations to.', layout=Layout(w…

Button(description='Congrats! COCO 2014 annotataions is downloaded!', disabled=True, layout=Layout(width='30%'…

Button(description='Congrats! COCO 2017 annotataions is downloaded!', disabled=True, layout=Layout(width='30%'…

Output()

## 4. Download and run Redis Gears Cluster Docker image
Run the cell below to download a Redis Gears Cluster Docker image (~605 MB), if not already present, and run it. 

In [4]:
redis_gears_cluster_image = "redislabs/rgcluster:1.0.6"
redis_gears_cluster_container_name = "demo_gears_cluster"

redis_gears_single_image = "redislabs/redisgears:1.0.6"
redis_gears_single_container_name = "demo_gears_single"

# Get the correct Redis Gears Images
!docker pull {redis_gears_single_image}
!docker pull {redis_gears_cluster_image}

# Check if the single container is already running.
container_info = !docker container inspect {redis_gears_single_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears single instance")
    !docker run --name {redis_gears_single_container_name} --rm -d -p 6379:6379 {redis_gears_single_image}


# Check if the cluster container is already running.
container_info = !docker container inspect {redis_gears_cluster_container_name}
if container_info[0] == "[]":
    print("Starting Redis Gears cluster instance")
    !docker run --name {redis_gears_cluster_container_name} --rm -d -p 30001:30001 -p 30002:30002 -p 30003:30003 {redis_gears_cluster_image}

print("Redis Gears containers are running!")

1.0.6: Pulling from redislabs/redisgears
Digest: sha256:ab126c449864cc9bc1b1facae26069c51d5f56760fb15b7a01371317fc69c17f
Status: Image is up to date for redislabs/redisgears:1.0.6
docker.io/redislabs/redisgears:1.0.6
1.0.6: Pulling from redislabs/rgcluster
Digest: sha256:563b2bf890085dc0b451de536fb7bacf8bfb98b6d1a162fda92f3b4742c4fd23
Status: Image is up to date for redislabs/rgcluster:1.0.6
docker.io/redislabs/rgcluster:1.0.6
Redis Gears containers are running!


## 5. Load Annotation Data into Redis cluster
By running the cell below, the COCO annotations downloaded above will be loaded into the Redis Cluster.

In [5]:
import glob
import itertools
import json
import os
import re
import redgrease
import zipfile

annotation_archive_files = os.path.join(download_location.value, "annotations_trainval*.zip")
annotation_archives = glob.glob(annotation_archive_files)

if not annotation_archives:
    print("no archives")
    raise SystemExit("Please download either or both COCO annotations as per instructions above.")

r = redgrease.RedisGears(host="localhost", port=30001)

annotation_json_pattern = re.compile("annotations/(\w+)_(\w+).json")

annotation_types = ["instances"] #  , "person_keypoints", "captions"]
datasets = ["val2014"]  # , "train2014", "val2017", "train2017"]

output = widgets.Output()
progress = widgets.Text("", layout=layout)

def load_annotation_info(info, dataset_name, annotation_type):
    annotation_info_key = f"/dataset/COCO/annotations/{annotation_type}/{dataset_name}/info"
    r.hset(annotation_info_key, mapping=info)
    return annotation_info_key

def load_license_info(license):
    license_key = f"/license/{license['id']}"
    if not r.exists(license_key):
        r.hset(license_key, mapping=license)
    return license_key

def load_image_info(image_info):
    img_info_key = f"/dataset/COCO/image/{image_info['id']}/info"
    if not r.exists(img_info_key):
        r.hset(img_info_key, mapping=image_info)
    return img_info_key

def load_keypoint_names(base_key, keypoints):
    keypoints_key = f"{base_key}/keypoints"
    r.lpush(keypoints_key, *keypoints)
    return keypoints_key

def load_list_of_str(base_key, sequence):
    list_key = f"{base_key}/skeleton"
    r.lpush(list_key, *map(str, sequence))
    return list_key

def load_category(category):
    category_key = f"/dataset/COCO/annotations/category/{category['id']}"

    if "keypoints" in category:
        category["keypoints"] = load_keypoint_names(category_key, category["keypoints"])
    if "skeleton" in category:
        category["skeleton"] = load_list_of_str(category_key, category["skeleton"])

    r.hset(category_key, mapping=category)
    return category_key

def load_segmentation(annotation_key, segmentation):
    segmentation_key = f"{annotation_key}/segmentation"
    if not r.exists(segmentation_key):
        for i, segment in enumerate(segmentation):
            segment_key = f"{segmentation_key}/{i}"
            r.lpush(segment_key, *segment)
            r.rpush(segmentation_key, segment_key)
    return segmentation_key

def load_annotation(annotation, dataset_name, annotation_type):
    annotation_key = f"/dataset/COCO/annotations/{annotation_type}/{dataset_name}/annotation/{annotation['id']}"
    
    if not r.exists(annotation_key):
        if "segmentation" in annotation:
            # Replace the 'segmentation' list-of-lists, with a key with a list of keys, that in turn point to the inner lists :)
            annotation["segmentation"] = load_segmentation(annotation_key, annotation["segmentation"])
        
        if "bbox" in annotation:
            # Replace the 'bbox' with a string reepresentaton.load_segmentation
            annotation["bbox"] = str(annotation["bbox"])

        if "keypoints" in annotation:
            annotation["keypoints"] = load_list_of_str(annotation_key, annotation["keypoints"])

        r.hset(annotation_key, mapping=annotation)
    return annotation_key

def load_annotation_jsons_from_zip(zip_file):
    with zipfile.ZipFile(zip_file) as archive:
        for file_name in archive.namelist():
            is_annotation_file = annotation_json_pattern.match(file_name)
            if not is_annotation_file:
                continue
            
            annotation_type = is_annotation_file.group(1)
            dataset_name = is_annotation_file.group(2)

            if not annotation_type in annotation_types:
                continue
            
            if not dataset_name in datasets:
                continue

            with archive.open(file_name) as json_file:
                contents = json.load(json_file)

            # info
            if "info" in contents:
                progress.value = f"Loading info for {dataset_name} {annotation_type}"
                load_annotation_info(contents["info"], dataset_name, annotation_type)

            # licenses
            if "licenses" in contents:
                progress.value = f"Loading licenses for {dataset_name} {annotation_type}"
                for lic in contents["licenses"]:
                    load_license_info(lic)
            
            # images
            if "images" in contents:
                progress.value = f"Loading images for {dataset_name} {annotation_type}"
                for image_info in contents["images"]:
                    load_image_info(image_info)

            # annotations
            if "annotations" in contents:
                progress.value = f"Loading annotations for {dataset_name} {annotation_type}"
                for annotation in contents["annotations"]:
                    load_annotation(annotation, dataset_name, annotation_type)

            # categories (for "instances" and "person_keypoints")
            if "categories" in contents:
                progress.value = f"Loading categories for {dataset_name} {annotation_type}"
                for category in contents["categories"]:
                    load_category(category)

            
display(progress)
for archive in annotation_archives:
    progress.value = f"Unzipping {archive}"
    load_annotation_jsons_from_zip(archive)
progress.value = "Done!"


Text(value='', layout=Layout(width='30%'))

# Demos
This is the actual Demo section. Everything above is just preparations.

1. [The Basics](#1.-The-Basics)
2. [Simple Analytics Query](#2.-Simple-Analytics-Query)
3. [Transaction Stream Processing](#3.-Transaction-Stream-Processing)
4. [Custom Command](#4.-Custom-Command)


<a id="demm-basics"></a>
## 1. The Basics
Showcasing some of the basic features and commands of the redgrease package.

In [124]:
import redgrease
import redgrease.utils

# Create connection / client for single instance Redis
single = redgrease.RedisGears() 

# Create connectoin / client for Redis Cluster 
cluster = redgrease.RedisGears(
    startup_nodes=[
        {"host":"localhost", "port":30001},
        {"host":"localhost", "port":30002},
        {"host":"localhost", "port":30003},
    ]
)


In [13]:
# The usual Redis commands work as expected
a = single.flushall()
b = single.set("Foo", 21)
c = single.hset("Bar", mapping={"spam":"eggs", "meaning":8})
d = single.hincrby("Bar", "meaning", 34)
e = single.xadd("tlogs::0", {"msg":"START", "from":0, "to":0, "amount":0})

a, b, c, d, e

(True, True, 2, 42, b'1616408947278-0')

In [44]:
# Gears commands can be accessed through the 'gears' attribute

cluster_pystats = cluster.gears.pystats()

print(f"Cluster Redis - Python Stats:\n{cluster_pystats}\n")



cluster_info = cluster.gears.infocluster()

print(f"Cluster Redis - Cluster Info:\n{cluster_info}\n")



cluster_refreshed = cluster.gears.refreshcluster()

print(f"Cluster Redis - Cluster Refresh Response:\n{cluster_refreshed}\n")



single_pystats = single.gears.pystats()

print(f"Single-node Redis - Python Stats:\n{single_pystats}\n")


# Iterate through all Redis key-value records, and return all record data.
all_records_gear = single.gears.pyexecute("GearsBuilder().run()")

print("Single-node Redis - All-records gear: [")
for result in all_records_gear:
    print(f"  {result}")
print("]\n")


# Iterate through all Redis key-value records, and return just the key and type
key_type_gear = single.gears.pyexecute("GearsBuilder().map(lambda record:(record['key'], record['type'])).run()")


print("Single-node Redis - Key-types gear: [")
for result in key_type_gear:
    print(f"  {result}")
print("]\n")


single_record_count = single.gears.pyexecute("GearsBuilder().count().run()")
print(f"Single-node Redis - Record count: {single_record_count}")

Cluster Redis - Python Stats:
PyStats(TotalAllocated=1930403157, PeakAllocated=12068267, CurrAllocated=11994022)

Cluster Redis - Cluster Info:
ClusterInfo(my_id='39db0e6c8869551606c5e2afe50948dcc06b3572', my_run_id='0d7711901c0a83e95decf21a3e20a76ece2e3b51', shards=[ShardInfo(id='a79198c55e6162cef83d2af625f9473e9a2e653d', ip='172.17.0.3', port=30003, unixSocket='None', runid='40b53913a81530d73b305ebed5f393c9ed564e25', minHslot=10923, maxHslot=16383, pendingMessages=0), ShardInfo(id='4abe400cd18ba4392bc4b69ce93d7eb39748ebc7', ip='172.17.0.3', port=30002, unixSocket='None', runid='ad166f6cec8e115053909c8d20801a3155ca2362', minHslot=5461, maxHslot=10922, pendingMessages=0), ShardInfo(id='39db0e6c8869551606c5e2afe50948dcc06b3572', ip='172.17.0.3', port=30001, unixSocket='None', runid='0d7711901c0a83e95decf21a3e20a76ece2e3b51', minHslot=0, maxHslot=5460, pendingMessages=0)])

Cluster Redis - Cluster Refresh Response:
True

Single-node Redis - Python Stats:
PyStats(TotalAllocated=43132814, 

### Instead of python functions as strings
RedGrease allows for cunstruction of GearFuntion objects

In [196]:
record_count = redgrease.KeysOnlyReader().count().run()

cluster_record_count = cluster.gears.pyexecute(record_count)

print(f"Cluster Redis - Total records: {cluster_record_count}")


# GearFunctions lets you compose and reuse partial functions
images = redgrease.KeysReader("/dataset/COCO/image/*/info").map(lambda record: record['value'])

image_count = images.count()

portrait_images = images.filter(lambda img: int(img['height']) > int(img['width']))

some_image_urls = portrait_images.collect().limit(4).map(lambda record: record['coco_url'])


# Different ways of running
img_cnt = image_count.run(on=cluster)
img_urls = cluster.gears.pyexecute(some_image_urls)
img_portrait = some_image_urls.run().on(cluster)


print(f"Total number of images: {img_cnt}/n")

from IPython.display import Image
from IPython.core.display import HTML 

print(f"Some portrait images")
for img_url in img_urls:
    display(Image(url=img_url))





Cluster Redis - Total records: 959243
Total number of images: 40504/n
Some portrait images


<a id="demo-query"></a>
## 2. Simple Analytics Query 

In [141]:
def dict_merge(d1, d2):
    return {**d1, **d2}

get_categories = (
    redgrease.KeysReader("/dataset/COCO/annotations/category/*")
    .map(lambda record: {record['value']['name']:record['value']['id']})
    .aggregate({},dict_merge, dict_merge)
)
                         
category_id_lookup = cluster.gears.pyexecute(get_categories)
category_id_lookup

{'cell phone': '77',
 'giraffe': '25',
 'tv': '72',
 'bed': '65',
 'toothbrush': '90',
 'vase': '86',
 'scissors': '87',
 'snowboard': '36',
 'motorcycle': '4',
 'tie': '32',
 'pizza': '59',
 'donut': '60',
 'hot dog': '58',
 'parking meter': '14',
 'dog': '18',
 'keyboard': '76',
 'laptop': '73',
 'sandwich': '54',
 'potted plant': '64',
 'spoon': '50',
 'traffic light': '10',
 'cake': '61',
 'tennis racket': '43',
 'refrigerator': '82',
 'truck': '8',
 'cup': '47',
 'cow': '21',
 'wine glass': '46',
 'boat': '9',
 'orange': '55',
 'fire hydrant': '11',
 'skateboard': '41',
 'knife': '49',
 'banana': '52',
 'sports ball': '37',
 'sheep': '20',
 'microwave': '78',
 'person': '1',
 'horse': '19',
 'airplane': '5',
 'dining table': '67',
 'bird': '16',
 'bowl': '51',
 'mouse': '74',
 'hair drier': '89',
 'suitcase': '33',
 'broccoli': '56',
 'sink': '81',
 'umbrella': '28',
 'toilet': '70',
 'couch': '63',
 'bench': '15',
 'zebra': '24',
 'clock': '85',
 'surfboard': '42',
 'bear': '23',

In [189]:

instance_annotations = (
    redgrease.KeysReader("/dataset/COCO/annotations/instances/*/annotation/*")
    .filter(lambda record: record['type'] == "hash")
    .map(lambda record: record["value"])
)



def accumulate_categories(image_id, accumulator, annotation):
    annotation_category_id = annotation['category_id']
    accumulator[annotation_category_id] = accumulator.get(annotation_category_id, 0) + 1
    return accumulator


def accumulate_category_counts(image_id, accumulator, category_count):
    for category, count in category_count.items():
        accumulator[category] = accumulator.get(category, 0) + count
    return accumulator


def format_img_stats(img_stats):
    return {
        'image_id' : img_stats['key'],
        'instances' : { cat_id:count for cat_id, count in img_stats['value'].items()}
    }

category_count_by_image = instance_annotations.aggregateby(
    extractor = lambda annotation : annotation['image_id'],
    zero = {},
    seqOp = accumulate_categories,
    combOp = accumulate_category_counts
).map(format_img_stats).collect()


cluster.gears.pyexecute(category_count_by_image.limit(2))






[{'image_id': '542676', 'instances': {'1': 1, '19': 1}},
 {'image_id': '396903', 'instances': {'1': 2, '5': 1}}]

In [228]:
def contstrain(constraints):
    # return a predicate for instance counts where the count of a set of categories
    # Constraints is a dict from category name to a tuple of (min_count, max_count)
    
    id_constraints = { category_id_lookup[cat_name]:x for cat_name, x in constraints.items()}
    
    def predicate(record):
        instances = record['instances']
        
        for cat_id, constraint in id_constraints.items():
            min_count, max_count = constraint
            
            if cat_id not in instances:
                if min_count is not ... and min_count > 0:
                    return False
                continue
            
            if min_count is not ... and instances[cat_id] < min_count:
                return False
            
            if max_count is not ... and instances[cat_id] > max_count:
                return False
            
        return True

    return predicate
    
query_params = {
    'person': [1, ...],
    'bird': [2, 2],
    'car' : [..., 0]
}


query = category_count_by_image.filter(contstrain(query_params))

query_result = cluster.gears.pyexecute(query.limit(10))

query_result



[{'image_id': '102843', 'instances': {'1': 14, '9': 1, '16': 2}},
 {'image_id': '172271', 'instances': {'1': 8, '9': 7, '16': 2, '15': 1}},
 {'image_id': '575719',
  'instances': {'1': 2, '34': 1, '15': 1, '16': 2, '9': 1}},
 {'image_id': '39081', 'instances': {'16': 2, '1': 1, '18': 1}},
 {'image_id': '281625',
  'instances': {'1': 14, '16': 2, '31': 1, '27': 1, '85': 2, '15': 3}},
 {'image_id': '478518', 'instances': {'16': 2, '47': 1, '1': 1}},
 {'image_id': '459733', 'instances': {'1': 1, '42': 1, '18': 1, '16': 2}},
 {'image_id': '470536', 'instances': {'1': 6, '15': 4, '16': 2}},
 {'image_id': '52435', 'instances': {'1': 14, '16': 2, '42': 2, '28': 1}},
 {'image_id': '230679',
  'instances': {'28': 4, '1': 1, '16': 2, '24': 1, '25': 1, '31': 1}}]

In [229]:

image_ids = [result['image_id'] for result in query_result]

get_image_urls = (
    images.filter(lambda img: img['id'] in image_ids)
    .map(lambda img: img["coco_url"])
)

image_urls = cluster.gears.pyexecute(get_image_urls)

for image_url in image_urls:
    display(Image(url=image_url))


<a id="demo-stream"></a>
## 3. Transaction Stream Processing

In [239]:
smtg = redgrease.StreamReader("tlogs:*").foreach(lambda x : redgrease.cmd.set("TEST", str(x))).register()

smtg.on(single)



ExecutionResult[bool](True)

In [240]:
single.xadd("tlogs::0", {"msg":"START", "from":0, "to":0, "amount":0})

b'1616580178324-0'

In [248]:
single.gears.dumpregistrations()

TypeError: __init__() got an unexpected keyword argument 'status'

In [249]:
import redis
r = redis.Redis()
r.execute_command("RG.DUMPREGISTRATIONS")

[[b'id',
  b'0000000000000000000000000000000000000000-10',
  b'reader',
  b'CommandReader',
  b'desc',
  None,
  b'RegistrationData',
  [b'mode',
   b'async',
   b'numTriggered',
   1,
   b'numSuccess',
   1,
   b'numFailures',
   0,
   b'numAborted',
   0,
   b'lastError',
   None,
   b'args',
   [b'trigger', b'EXEC']],
  b'PD',
  b"{'sessionId':'0000000000000000000000000000000000000000-9', 'depsList':[{'name':'redgrease[runtime]', 'basePath':'/var/opt/redislabs/modules/rg//python3_1.0.6//redgrease[runtime]', 'wheels':['packaging-20.9-py2.py3-none-any.whl','pyparsing-2.4.7-py2.py3-none-any.whl','cloudpickle-1.6.0-py3-none-any.whl','redis-3.5.3-py2.py3-none-any.whl','redgrease-0.1.28-py3-none-any.whl','wrapt-1.12.1-py3-none-any.whl','attrs-20.3.0-py2.py3-none-any.whl']}]}"],
 [b'id',
  b'0000000000000000000000000000000000000000-8',
  b'reader',
  b'CommandReader',
  b'desc',
  None,
  b'RegistrationData',
  [b'mode',
   b'async',
   b'numTriggered',
   2,
   b'numSuccess',
   2,
   b'n

In [242]:
single.get("TEST")

<a id="demo-command"></a>
## 4. Custom Command

In [135]:
print("Demo 4 - Custom Command")
cluster = redgrease.RedisGears(
    startup_nodes=[
        {"host":"localhost", "port":30001},
        {"host":"localhost", "port":30002},
        {"host":"localhost", "port":30003},
    ]
)
cluster.hgetall("/dataset/COCO/annotations/category/47")

Demo 4 - Custom Command


{b'supercategory': b'kitchen', b'id': b'47', b'name': b'cup'}